In [673]:
% reset -f

In [116]:
from numpy.random import seed
seed(1)

In [117]:
import nltk
import gensim
import pandas as pd

data=pd.read_table("//QATLPCFS001/Users/akishore/Desktop/DL/voc/articles.txt",encoding='latin1')
keep_subcats=pd.read_csv("//QATLPCFS001/Users/akishore/Desktop/DL/voc/keep_subcats.csv",encoding='latin1')
data.head()

,ARTICLE_NAME,SUBCAT_NAME
0,Yoplait Berry Delights 6pk,DAIRY - YOGHURT
1,Heat & Eat Green Curry Small,SALADS
2,Emer Lds Tall Boot Harper 6 11 Tan PP,FAMILY FOOTWEAR
3,Beef Topside Cap On Ctn,FRESH BEEF SUPPLIES CTN
4,Natures Way Mini Multi 50 + 50s,VITAMINS


In [118]:
import numpy as np
seed(1)
np.random.choice(10,10,replace=False)

array([2, 9, 6, 4, 0, 3, 1, 7, 8, 5])

In [119]:
# data2=data
# data2.head()
# data2=data.loc[np.random.choice(data.shape[0],data.shape[0],replace=False)]
# data2.shape

In [120]:
# data2=data.reset_index()
# data2=data2.drop(['index'],axis=1)

In [121]:
# data3=data2
# data3['ARTICLE_NAME']=(data['ARTICLE_NAME']+" "+data2['ARTICLE_NAME'])

In [122]:
# data3.head()

In [123]:
# data=data.append(data3)
# data.shape

In [124]:
data=pd.merge(data,keep_subcats,on="SUBCAT_NAME",how='left')
data.shape

(360029, 3)

In [125]:
data.head()

,ARTICLE_NAME,SUBCAT_NAME,KEEP_SUBCAT
0,Yoplait Berry Delights 6pk,DAIRY - YOGHURT,1.0
1,Heat & Eat Green Curry Small,SALADS,NaN
2,Emer Lds Tall Boot Harper 6 11 Tan PP,FAMILY FOOTWEAR,NaN
3,Beef Topside Cap On Ctn,FRESH BEEF SUPPLIES CTN,NaN
4,Natures Way Mini Multi 50 + 50s,VITAMINS,NaN


In [126]:
import numpy as np
data['SUBCAT_NAME']=np.where(data['KEEP_SUBCAT']==1,data['SUBCAT_NAME'],"unknown")
data.head()

,ARTICLE_NAME,SUBCAT_NAME,KEEP_SUBCAT
0,Yoplait Berry Delights 6pk,DAIRY - YOGHURT,1.0
1,Heat & Eat Green Curry Small,unknown,NaN
2,Emer Lds Tall Boot Harper 6 11 Tan PP,unknown,NaN
3,Beef Topside Cap On Ctn,unknown,NaN
4,Natures Way Mini Multi 50 + 50s,unknown,NaN


In [127]:
data_subcats=data[data['SUBCAT_NAME']!="unknown"]
data_subcats.shape

(83570, 3)

In [128]:
data_unknown=data[data['SUBCAT_NAME']=="unknown"]
data_unknown.shape

(276459, 3)

In [129]:
data_unknown=data_unknown.reset_index()
data_unknown=data_unknown.drop(['index'],axis=1)
random_array=np.random.choice(data_unknown.shape[0],100000,replace=False)
data_unknown2=data_unknown.loc[random_array]
data_unknown2.head()
data_unknown2=data_unknown2.reset_index()
data_unknown2=data_unknown2.drop(['index'],axis=1)
data_unknown2.head()

,ARTICLE_NAME,SUBCAT_NAME,KEEP_SUBCAT
0,DVD Teenage Mutant Ninja Turtles,unknown,NaN
1,Taste of Asia Sweet Chilli Sauce 700ml,unknown,NaN
2,Smitten - 12 Stories,unknown,NaN
3,FRESH AS OREGANO POWDER 6G,unknown,NaN
4,Bk Japanese Phrs 7 Japan 7RRP 14.99,unknown,NaN


In [130]:
data=data_subcats.append(data_unknown2)
data.shape

(183570, 3)

In [131]:
#%reset -f

In [132]:
data=data.drop(["KEEP_SUBCAT"],axis=1)
data.shape

(183570, 2)

In [133]:
data=data.reset_index()
data=data.drop(['index'],axis=1)

In [134]:
import re
from nltk.corpus import stopwords
from inflection import singularize
def preprocess(text):
    text=text.lower()
    text=re.sub("[^a-zA-Z]+"," ",text)
    text2=text.split()
    #filtered_words = [word for word in text2 if word not in stopwords.words('english')]
    filtered_words2 = [word for word in text2 if word not in ['kg','ml','ea','pk','cm','gm','or','the','in','and','is','of','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won','stock','buy','products','store','shelf','get','brand','always','go','one','time','shop','range','like','brands','limited','low','times','day','free','never','sale','shopping','last','food','variety','choice','back','two','home','good','small','run','found','well','use','something','still','first','number','full','days','less','selection','next','little','months','select','home','brand','special','price','produce','quality','regular','per','size','stocking','better','long','pm','hard','made','make','new','night','morning','way','late','reduced','space','great','display','three','normal','stuff','non','bare','may','place','away','dont','saturday','ever','happy','us','top','feel','take','finding','filled','wish','friday','miss','floor','nice','row','done','start','litter','pick','quick','right','open','reach','fill','online','note','yes','spots','walk','room','cleaning','system','easy','husband','ripe','truck','cases','tags','serve','centre','th','dates','grown','labels','company','key','hour','wife','simple','waste','selections','cabinet','deal','call','fine','inside','along','crushed','felt','lost','wonder','rare','grab','budget','added','trays','six','rush','push','oh','collect','primary','refill','general','previous','superior','always','homebrand','range','cosmetics','finish','dinner','lucky','tourist','season','part','variety','city','happy','answer','previous','see','se','really','bad','shop','complete','max','whole','short','check','out','spot','basic','frozen','business','essential','shelves','child','ad','brief','favourite','pass','note','direct','woolworths','bc','ones','delete','popular','overseas','sourced','mum','often','items','everyday','son','page','break','dozen','half','australian','aussie','mixed','show','set','wide','st','fresh','much','lines','box','end','self','big','nothing','large','local','work','stick','flavours','flavour','date','bit','paper','high','isles','aisle','simply','fresh','best','travel','standards','standard','christmas','foods','old','remote','drive','live','1st','wide','families','extra','four','pack','cut','label','ww','park','house','ready','single','road','flowers','favourite','favourites','australia','fast','hill','love','previous','yrs','sock','wash','training','mths','esp','healthy','gripe','thigh','middle']]
    text3=' '.join([singularize(word) for word in filtered_words2 if len(word) >1])
    text4=text3.split()
    filtered_words3 = [word for word in text4 if word not in ['kg','ml','ea','pk','cm','gm','or','the','in','and','is','of','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won','stock','buy','products','store','shelf','get','brand','always','go','one','time','shop','range','like','brands','limited','low','times','day','free','never','sale','shopping','last','food','variety','choice','back','two','home','good','small','run','found','well','use','something','still','first','number','full','days','less','selection','next','little','months','select','home','brand','special','price','produce','quality','regular','per','size','stocking','better','long','pm','hard','made','make','new','night','morning','way','late','reduced','space','great','display','three','normal','stuff','non','bare','may','place','away','dont','saturday','ever','happy','us','top','feel','take','finding','filled','wish','friday','miss','floor','nice','row','done','start','litter','pick','quick','right','open','reach','fill','online','note','yes','spots','walk','room','cleaning','system','easy','husband','ripe','truck','cases','tags','serve','centre','th','dates','grown','labels','company','key','hour','wife','simple','waste','selections','cabinet','deal','call','fine','inside','along','crushed','felt','lost','wonder','rare','grab','budget','added','trays','six','rush','push','oh','collect','primary','refill','general','previous','superior','always','homebrand','range','cosmetics','finish','dinner','lucky','tourist','season','part','variety','city','happy','answer','previous','see','se','really','bad','shop','complete','max','whole','short','check','out','spot','basic','frozen','business','essential','shelves','child','ad','brief','favourite','pass','note','direct','woolworths','bc','ones','delete','popular','overseas','sourced','mum','often','items','everyday','son','page','break','dozen','half','australian','aussie','mixed','show','set','wide','st','fresh','much','lines','box','end','self','big','nothing','large','local','work','stick','flavours','flavour','date','bit','paper','high','isles','aisle','simply','fresh','best','travel','standards','standard','christmas','foods','old','remote','drive','live','1st','wide','families','extra','four','pack','cut','label','ww','park','house','ready','single','road','flowers','favourite','favourites','australia','fast','hill','love','previous','yrs','sock','wash','training','mths','esp','healthy','gripe','thigh','middle']]
    text5=' '.join([singularize(word) for word in filtered_words3 if len(word) >1])
    return(text5)

In [135]:
data.shape

(183570, 2)

In [136]:
comments=pd.read_table("//QATLPCFS001/Users/akishore/Desktop/DL/voc/comments.csv",encoding='latin1')

In [137]:
def comments_preprocess(text):
    text=text.lower()
    text=re.sub("[^a-zA-Z]+"," ",text)
    text2=text.split()
    #filtered_words = [word for word in text2 if word not in stopwords.words('english')]
    filtered_words2 = [word for word in text2 if word not in ['kg','ml','ea','pk','cm','gm','or','the','in','and','is','of','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won','stock','buy','products','store','shelf','get','brand','always','go','one','time','shop','range','like','brands','limited','low','times','day','free','never','sale','shopping','last','food','variety','choice','back','two','home','good','small','run','found','well','use','something','still','first','number','full','days','less','selection','next','little','months','select','home','brand','primary','refill','general','previous','superior','always','homebrand','range','cosmetics','finish','dinner','lucky','tourist','season','part','variety','city','happy','answer','previous','see','se','really','bad','shop','complete','max','whole','short','check','out','spot','basic','frozen','business','essential','shelves','child','ad','brief','favourite','pass','note','direct','woolworths','bc']]
    text3=' '.join([word for word in filtered_words2 if len(word) >1])
    text4=text3.split()
    filtered_words3 = [word for word in text4 if word not in ['kg','ml','ea','pk','cm','gm','or','the','in','and','is','of','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won','stock','buy','products','store','shelf','get','brand','always','go','one','time','shop','range','like','brands','limited','low','times','day','free','never','sale','shopping','last','food','variety','choice','back','two','home','good','small','run','found','well','use','something','still','first','number','full','days','less','selection','next','little','months','select','home','brand','special','price','produce','quality','regular','per','size','stocking','better','long','pm','hard','made','make','new','night','morning','way','late','reduced','space','great','display','three','normal','stuff','non','bare','may','place','away','dont','saturday','ever','happy','us','top','feel','take','finding','filled','wish','friday','miss','floor','nice','row','done','start','litter','pick','quick','right','open','reach','fill','online','note','yes','spots','walk','room','cleaning','system','easy','husband','ripe','truck','cases','tags','serve','centre','th','dates','grown','labels','company','key','hour','wife','simple','waste','selections','cabinet','deal','call','fine','inside','along','crushed','felt','lost','wonder','rare','grab','budget','added','trays','six','rush','push','oh','collect','primary','refill','general','previous','superior','always','homebrand','range','cosmetics','finish','dinner','lucky','tourist','season','part','variety','city','happy','answer','previous','see','se','really','bad','shop','complete','max','whole','short','check','out','spot','basic','frozen','business','essential','shelves','child','ad','brief','favourite','pass','note','direct','woolworths','bc','ones','delete','popular','overseas','sourced','mum','often','items','everyday','son','page','break','dozen','half','australian','aussie','mixed','show','set','wide','st','fresh','much','lines','box','end','self','big','nothing','large','local','work','stick','flavours','flavour','date','bit','paper','high','isles','aisle','simply','fresh','best','travel','standards','standard','christmas','foods','old','remote','drive','live','1st','wide','families','extra','four','pack','cut','label','ww','park','house','ready','single','road','flowers','favourite','favourites','australia','fast','hill','love']]
    text5=' '.join([singularize(word) for word in filtered_words3 if len(word) >1])
    return(text5)

In [138]:
comments.columns

Index(['Comment'], dtype='object')

In [139]:
comments['Comment']=comments['Comment'].apply(comments_preprocess)
count_comments=comments['Comment'].str.split(' ', expand=True).stack().value_counts()
count_comments2=pd.DataFrame(count_comments)
count_comments2=count_comments2.reset_index()
count_comments2.columns=["word","Freq"]
count_comments2.head()

,word,Freq
0,available,13497
1,empty,6314
2,wanted,5765
3,bread,4782
4,milk,4717


In [173]:
count_comments2.to_csv("//QATLPCFS001/Users/akishore/Desktop/DL/voc/comment_word_counts8.csv",index=False)

In [140]:
data['ARTICLE_NAME']=data['ARTICLE_NAME'].apply(preprocess)

In [141]:
data.head()

,ARTICLE_NAME,SUBCAT_NAME
0,yoplait berry delight,DAIRY - YOGHURT
1,destiny playstatpre order,PET NEEDS - DOG TREATS
2,disney plush,BABY - BASICS
3,gold alula newborn,BABY FORMULA
4,rowy caramel fudge,CHEESE PRE-PACKED ENTERTAINING


In [142]:
data[data['SUBCAT_NAME'].str.contains("MILK")]['SUBCAT_NAME'].value_counts()

MILK - FLAVOURED              809
DAIRY - MILK                  546
LONGLIFE MILK & SOY DRINKS    423
MILK ADDITIVES                291
Name: SUBCAT_NAME, dtype: int64

In [143]:
 counts=data['ARTICLE_NAME'].str.split(' ', expand=True).stack().value_counts()

In [144]:
counts['item']

14

In [145]:
counts2=pd.DataFrame(counts)
counts2=counts2.reset_index()
counts2.columns=["word","Freq"]
counts2.head()

,word,Freq
0,dvd,4762
1,chicken,3620
2,black,3403
3,card,3306
4,cheese,3068


In [146]:
counts2['Freq'].describe()

count    58540.000000
mean        12.026375
std         76.567437
min          1.000000
25%          1.000000
50%          1.000000
75%          4.000000
max       4762.000000
Name: Freq, dtype: float64

In [147]:
counts3=counts2[counts2['Freq']>50]
counts3.shape

(2288, 2)

In [1175]:
counts3.to_csv("//QATLPCFS001/Users/akishore/Desktop/DL/voc/article_word_counts7.csv",index=False)

In [148]:
len(counts3)

2288

In [149]:
all_prods=list(data['ARTICLE_NAME'])
t=data['ARTICLE_NAME'].tolist()
t2=t
for i in range(len(t)):
    t2[i]=t[i].split()

In [150]:
from gensim.models import Phrases
bigram = Phrases(t2, min_count=50, threshold=2)

2017-11-13 17:04:33,942 : INFO : collecting all words and their counts
2017-11-13 17:04:33,944 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2017-11-13 17:04:34,044 : INFO : PROGRESS: at sentence #10000, processed 29571 words and 30908 word types
2017-11-13 17:04:34,146 : INFO : PROGRESS: at sentence #20000, processed 59268 words and 52666 word types
2017-11-13 17:04:34,248 : INFO : PROGRESS: at sentence #30000, processed 89023 words and 70940 word types
2017-11-13 17:04:34,350 : INFO : PROGRESS: at sentence #40000, processed 118590 words and 87322 word types
2017-11-13 17:04:34,454 : INFO : PROGRESS: at sentence #50000, processed 148040 words and 102325 word types
2017-11-13 17:04:34,556 : INFO : PROGRESS: at sentence #60000, processed 177456 words and 116030 word types
2017-11-13 17:04:34,658 : INFO : PROGRESS: at sentence #70000, processed 207196 words and 129392 word types
2017-11-13 17:04:34,766 : INFO : PROGRESS: at sentence #80000, processed 236810 words 

In [151]:
t3=[]
for i in range(len(t2)):
    sent=t2[i]
    t3.append(bigram[sent])

C:\Program Files\Anaconda3\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [152]:
t3[0:5]

[['yoplait', 'berry', 'delight'],
 ['destiny', 'playstatpre', 'order'],
 ['disney', 'plush'],
 ['gold', 'alula', 'newborn'],
 ['rowy', 'caramel', 'fudge']]

In [153]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 100    # Word vector dimensionality                      
min_word_count = 50   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 9         # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [154]:
from gensim.models import word2vec
print("Training model...")
w2v_model = word2vec.Word2Vec(t3, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

2017-11-13 17:04:43,794 : INFO : collecting all words and their counts
2017-11-13 17:04:43,795 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-11-13 17:04:43,815 : INFO : PROGRESS: at sentence #10000, processed 37416 words, keeping 8742 word types
2017-11-13 17:04:43,835 : INFO : PROGRESS: at sentence #20000, processed 74988 words, keeping 12950 word types
2017-11-13 17:04:43,855 : INFO : PROGRESS: at sentence #30000, processed 112586 words, keeping 16177 word types
2017-11-13 17:04:43,875 : INFO : PROGRESS: at sentence #40000, processed 150072 words, keeping 18882 word types
2017-11-13 17:04:43,895 : INFO : PROGRESS: at sentence #50000, processed 187472 words, keeping 21304 word types
2017-11-13 17:04:43,917 : INFO : PROGRESS: at sentence #60000, processed 224781 words, keeping 23374 word types
2017-11-13 17:04:43,942 : INFO : PROGRESS: at sentence #70000, processed 262413 words, keeping 25415 word types
2017-11-13 17:04:43,969 : INFO : PROGRESS: at sen

Training model...


2017-11-13 17:04:44,010 : INFO : PROGRESS: at sentence #100000, processed 372918 words, keeping 36653 word types
2017-11-13 17:04:44,030 : INFO : PROGRESS: at sentence #110000, processed 409285 words, keeping 40394 word types
2017-11-13 17:04:44,054 : INFO : PROGRESS: at sentence #120000, processed 445485 words, keeping 43536 word types
2017-11-13 17:04:44,086 : INFO : PROGRESS: at sentence #130000, processed 481527 words, keeping 46364 word types
2017-11-13 17:04:44,107 : INFO : PROGRESS: at sentence #140000, processed 517924 words, keeping 48936 word types
2017-11-13 17:04:44,129 : INFO : PROGRESS: at sentence #150000, processed 554260 words, keeping 51380 word types
2017-11-13 17:04:44,152 : INFO : PROGRESS: at sentence #160000, processed 590717 words, keeping 53804 word types
2017-11-13 17:04:44,178 : INFO : PROGRESS: at sentence #170000, processed 626921 words, keeping 55987 word types
2017-11-13 17:04:44,201 : INFO : PROGRESS: at sentence #180000, processed 663141 words, keeping 

In [155]:
w2v_model.most_similar("cheese")

2017-11-13 17:04:49,886 : INFO : precomputing L2-norms of word weight vectors


[('chse', 0.8616633415222168),
 ('mainland_cheese', 0.8574714660644531),
 ('cheddar', 0.8551862239837646),
 ('ch', 0.8419789671897888),
 ('mainland', 0.8169941902160645),
 ('grated', 0.7922428250312805),
 ('cheese_slouse', 0.785186767578125),
 ('salami', 0.7840813398361206),
 ('brie', 0.7780658602714539),
 ('gouda', 0.7772737145423889)]

In [156]:
data2=data
print(data2.shape)
data2.head()

(183570, 2)


,ARTICLE_NAME,SUBCAT_NAME
0,yoplait berry delight,DAIRY - YOGHURT
1,destiny playstatpre order,PET NEEDS - DOG TREATS
2,disney plush,BABY - BASICS
3,gold alula newborn,BABY FORMULA
4,rowy caramel fudge,CHEESE PRE-PACKED ENTERTAINING


In [157]:
ind=[]
dep=[]
ex=[]
word=[]
x=t3
t2=x
y=data2['SUBCAT_NAME'].tolist()
k=0
m=0
for i in range(len(x)):
    t2[i]=x[i]
    if(i%10000==0):
        print(i)
    #print(t2[i])
    for j in range(len(t2[i])):
        try:
            ind.append((w2v_model[t2[i][j]]))
            dep.append(y[i])
            ex.append(data2['ARTICLE_NAME'][i])
            word.append(t2[i][j])
            m=m+1
            #print(m)
        except KeyError:
            #print(\its ok\)
            k=k+1

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000


In [158]:
tt=0
tt=pd.DataFrame(ind)
tt.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.502269,0.065828,0.532839,-0.876406,-0.042052,-0.208942,-0.426184,0.461035,0.945602,0.214742,...,1.175375,-0.215694,-0.760396,0.294615,0.128994,-0.614231,-0.133625,-0.555341,-0.086292,0.309654
1,-0.627736,-0.113400,0.509135,-0.398738,-0.296475,0.163595,-0.656789,0.064620,0.628424,-0.282380,...,0.976393,-0.242951,-0.749223,0.369747,0.336394,-0.381900,0.028104,-0.491618,-0.303166,-0.256417
2,-0.249966,-0.197279,0.119820,-0.255025,0.279711,-0.285171,-0.348683,-0.050246,0.356724,0.118928,...,0.495194,0.111556,-0.284101,0.469044,0.164614,-0.334468,-0.006587,-0.322965,-0.071901,0.221618
3,0.204293,-0.047774,0.692463,0.268272,-0.148196,0.284098,0.049278,0.356313,0.605597,-0.414332,...,-0.103590,-0.549331,-0.670091,0.419861,-0.474965,-0.255029,-0.114241,-0.233007,-0.267427,-0.001939
4,0.314717,-0.226887,0.325792,0.142807,-0.105028,0.041693,0.014319,0.413398,0.319911,-0.220551,...,-0.134301,-0.196859,-0.522430,0.216962,-0.111961,-0.364099,-0.051591,-0.008306,-0.319499,0.140814


In [159]:
import numpy as np
tt['subcat']=dep
tt['article']=ex
tt['word']=word
tt['cheese']=np.where(((tt['subcat']=="DAIRY - CHEESE") | (tt['subcat']=="PASTA SAUCE & CHEESE") | (tt['subcat']=="CHEESE PRE-PACKED ENTERTAINING") | (tt['subcat']=="CHEESE PRE-PACKED COOKING") | (tt['subcat']=="CHEESE PRE-PACKED COOKING") | (tt['subcat']=="CHEESE BULK")),1.0,0.0)
tt['canned_fish']=np.where(tt['subcat']=="CANNED FISH",1.0,0.0)
tt['chips']=np.where(((tt['subcat']=="CHIPS - SINGLE SERVE") | (tt['subcat']=="CHIPS - SHARING") | (tt['subcat']=="CHIPS - MULTIPACKS")),1.0,0.0)
tt.columns

Index([            0,             1,             2,             3,
                   4,             5,             6,             7,
                   8,             9,
       ...
                  96,            97,            98,            99,
            'subcat',     'article',        'word',      'cheese',
       'canned_fish',       'chips'],
      dtype='object', length=106)

In [160]:
tt.head()

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,subcat,article,word,cheese,canned_fish,chips
0,-0.502269,0.065828,0.532839,-0.876406,-0.042052,-0.208942,-0.426184,0.461035,0.945602,0.214742,...,-0.133625,-0.555341,-0.086292,0.309654,DAIRY - YOGHURT,yoplait berry delight,yoplait,0.0,0.0,0.0
1,-0.627736,-0.113400,0.509135,-0.398738,-0.296475,0.163595,-0.656789,0.064620,0.628424,-0.282380,...,0.028104,-0.491618,-0.303166,-0.256417,DAIRY - YOGHURT,yoplait berry delight,berry,0.0,0.0,0.0
2,-0.249966,-0.197279,0.119820,-0.255025,0.279711,-0.285171,-0.348683,-0.050246,0.356724,0.118928,...,-0.006587,-0.322965,-0.071901,0.221618,DAIRY - YOGHURT,yoplait berry delight,delight,0.0,0.0,0.0
3,0.204293,-0.047774,0.692463,0.268272,-0.148196,0.284098,0.049278,0.356313,0.605597,-0.414332,...,-0.114241,-0.233007,-0.267427,-0.001939,BABY - BASICS,disney plush,disney,0.0,0.0,0.0
4,0.314717,-0.226887,0.325792,0.142807,-0.105028,0.041693,0.014319,0.413398,0.319911,-0.220551,...,-0.051591,-0.008306,-0.319499,0.140814,BABY - BASICS,disney plush,plush,0.0,0.0,0.0


In [161]:
tt2=pd.pivot_table(tt.drop(['subcat','word','cheese','canned_fish','chips'],axis=1),index=['article'],aggfunc=np.mean)

In [162]:
tt2=tt2.reset_index()
tt3=tt[['article','subcat']]
tt4=tt3.drop_duplicates()
tt5=pd.merge(tt2,tt4)
tt5.shape

(158292, 102)

In [163]:
tt_word=pd.merge(tt,counts2,on='word',how='inner')

In [164]:
tt_word.shape

(430524, 107)

In [165]:
tt.shape

(458085, 106)

In [166]:
tt_word=tt_word.sort(['article'])

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [167]:
tt_word=tt_word.reset_index()

In [168]:
tt_word=tt_word.drop(['Freq'],axis=1)
tt_word.shape

(430524, 107)

In [169]:
tt_word=tt_word.drop(['index'],axis=1)
tt_word.shape

(430524, 106)

In [170]:
tt_word2=pd.pivot_table(tt.drop(['subcat','word','cheese','canned_fish','chips'],axis=1),index=['article'],aggfunc=np.mean)

In [171]:
tt_word2=tt_word2.reset_index()
tt_word3=tt_word[['article','subcat']]
tt_word4=tt_word3.drop_duplicates()
tt_word5=pd.merge(tt_word2,tt_word4)
tt_word5.shape

(155474, 102)

In [172]:
tt6=tt5.append(tt_word5)

In [61]:
#tt6=tt5

In [173]:
tt6.shape

(313766, 102)

In [174]:
tt6['cheese']=np.where(((tt6['subcat']=="DAIRY - CHEESE") | (tt6['subcat']=="PASTA SAUCE & CHEESE") | (tt6['subcat']=="CHEESE PRE-PACKED ENTERTAINING") | (tt6['subcat']=="CHEESE PRE-PACKED COOKING") | (tt6['subcat']=="CHEESE PRE-PACKED COOKING") | (tt6['subcat']=="CHEESE BULK")),1.0,0.0)
tt6['canned_fish']=np.where(tt6['subcat']=="CANNED FISH",1.0,0.0)
tt6['chips']=np.where(((tt6['subcat']=="CHIPS - SINGLE SERVE") | (tt6['subcat']=="CHIPS - SHARING") | (tt6['subcat']=="CHIPS - MULTIPACKS")),1.0,0.0)
tt6['final_subcat']=np.where(tt6['cheese']==1.0,1,np.where(tt6['canned_fish']==1.0,2,np.where(tt6['chips']==1.0,3,0)))

In [175]:
tt6.columns

Index([     'article',              0,              1,              2,
                    3,              4,              5,              6,
                    7,              8,
       ...
                   95,             96,             97,             98,
                   99,       'subcat',       'cheese',  'canned_fish',
              'chips', 'final_subcat'],
      dtype='object', length=106)

In [176]:
tt6.head()

,article,0,1,2,3,4,5,6,7,8,...,95,96,97,98,99,subcat,cheese,canned_fish,chips,final_subcat
0,aa air freshener vanilla,-0.174764,-0.215149,0.324503,-0.244157,-0.143780,-0.090593,-0.252002,0.354626,0.304842,...,-0.327167,0.012143,-0.161202,-0.103043,0.060150,unknown,0.0,0.0,0.0,0
1,aa attum flour,0.090097,-0.187574,0.084583,-0.257561,0.049600,-0.161327,-0.113648,0.135219,0.152690,...,-0.299314,-0.036575,-0.162999,-0.099049,0.164816,unknown,0.0,0.0,0.0,0
2,aa order required,0.124038,-0.300707,0.124215,-0.083251,-0.061721,-0.042778,-0.046722,0.178745,0.261352,...,-0.204476,0.037098,-0.084059,-0.130874,0.157403,unknown,0.0,0.0,0.0,0
3,aapri scrub facial original,0.126395,-0.359556,0.093018,-0.063419,-0.173237,-0.147554,-0.192202,0.122712,0.236076,...,-0.235825,0.079171,-0.119677,-0.034439,0.139863,unknown,0.0,0.0,0.0,0
4,aashiayana chicken tikka marsala,0.414469,-0.818314,0.238988,-0.557760,0.500375,-0.058706,-0.174958,-0.242977,0.215147,...,-0.189325,0.411273,-0.418959,-0.368676,0.549393,FREEZER - MEALS,0.0,0.0,0.0,0


In [177]:
np.random.randint(0,5,1)[0]

0

In [178]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
tt6["make_code"] = lb_make.fit_transform(tt6["subcat"])
tt6[["subcat", "make_code"]].head(11)

,subcat,make_code
0,unknown,110
1,unknown,110
2,unknown,110
3,unknown,110
4,FREEZER - MEALS,53
5,FREEZER - MEALS,53
6,unknown,110
7,HEALTH FOODS,66
8,HEALTH FOODS,66
9,unknown,110


In [179]:
X=tt6.iloc[:,0:(num_features+1)]
print(X.shape)
y=tt6['make_code']
y.shape


(313766, 101)


(313766,)

In [180]:
# X=tt6.iloc[:,1:(num_features+1)]
# print(X.shape)
# y=tt6['subcat']
# y.shape

In [181]:
seed(1)
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=10)

In [182]:
# from sklearn.linear_model import LogisticRegression
# lm=LogisticRegression()


In [183]:
# lm.fit(x_train, y_train)

In [184]:
# pred=lm.predict(x_test)

In [185]:
# pd.Series(pred).value_counts()

In [186]:
# correct=np.where(pred==y_test,1,0)
# np.mean(correct)

In [187]:
y_test[0:5]

27787      21
154326     82
138253    110
33549     110
16984     110
Name: make_code, dtype: int64

In [188]:
seed(1)
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=10)

In [189]:
type(x_train)

pandas.core.frame.DataFrame

In [190]:
X_train=np.array(x_train)
y_train=np.array(y_train)

In [191]:
X_train=np.array(x_train.drop(['article'],axis=1))
y_train=np.array(y_train)

In [192]:
len(set(y))

111

In [193]:
import numpy as np
nb_classes = len(set(tt6['make_code']))

def indices_to_one_hot(data, nb_classes):
    """Convert an iterable of indices to one-hot encoded labels."""
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]

In [194]:
nb_classes

111

In [195]:
X_train=np.array(x_train.drop(['article'],axis=1))
y_train=np.array(y_train)

In [196]:
y_train[:3]

array([ 63,  64, 108], dtype=int64)

In [197]:
y_train2=indices_to_one_hot(y_train,nb_classes)

In [198]:
X_test=np.array(x_test.drop(['article'],axis=1))
y_test=np.array(y_test)
y_test2=indices_to_one_hot(y_test,nb_classes)

In [199]:
#y_test2=indices_to_one_hot(y_test1,nb_classes)

In [200]:
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

In [201]:
seed(1)
from keras.models import *
from keras.layers import *
# create model
model = Sequential()

model.add(Dense(1000, input_dim=num_features, activation='relu',init='glorot_uniform'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dense(nb_classes, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X_train, y_train2, epochs=10, batch_size=1024,validation_data=(X_test, y_test2),shuffle=False)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1000, kernel_initializer="glorot_uniform", activation="relu", input_dim=100)`


Train on 219636 samples, validate on 94130 samples
Epoch 1/10
219636/219636 [==============================] - 20s - loss: 2.2293 - acc: 0.5506 - val_loss: 1.6701 - val_acc: 0.5923
Epoch 2/10
219636/219636 [==============================] - 19s - loss: 1.6442 - acc: 0.5934 - val_loss: 1.5130 - val_acc: 0.6203
Epoch 3/10
219636/219636 [==============================] - 19s - loss: 1.5454 - acc: 0.6100 - val_loss: 1.4459 - val_acc: 0.6310
Epoch 4/10
219636/219636 [==============================] - 19s - loss: 1.4929 - acc: 0.6211 - val_loss: 1.4083 - val_acc: 0.6389
Epoch 5/10
219636/219636 [==============================] - 19s - loss: 1.4576 - acc: 0.6272 - val_loss: 1.3765 - val_acc: 0.6449
Epoch 6/10
219636/219636 [==============================] - 19s - loss: 1.4301 - acc: 0.6327 - val_loss: 1.3545 - val_acc: 0.6483
Epoch 7/10
219636/219636 [==============================] - 19s - loss: 1.4099 - acc: 0.6375 - val_loss: 1.3361 - val_acc: 0.6519
Epoch 8/10
219636/219636 [=============

In [575]:
# X_test1=X_test
# y_test1=y_test

In [202]:
predictions = model.predict(X_train)
predictions2=pd.DataFrame(predictions)
predictions3=np.argmax(predictions,axis=1)
X_train2=pd.DataFrame(X_train)
X_train2['prediction']=predictions3
X_train2['final_subcat']=y_train
X_train2.index=x_train.index
X_train2['article']=x_train['article']

In [203]:
predictions3[0:5]

array([110,  64, 108,  94,   4], dtype=int64)

In [204]:
pd.crosstab(X_train2['final_subcat'],X_train2['prediction'])

prediction,0,1,2,3,4,5,6,7,8,9,...,101,102,103,104,105,106,107,108,109,110
final_subcat,,,,,,,,,,,,,,,,,,,,,
0,402,0,2,0,7,0,0,0,5,12,...,2,6,0,3,0,0,9,20,19,1057
1,0,442,31,20,35,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1903
2,0,10,242,32,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,332
3,0,3,0,562,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,122
4,1,5,9,1,986,12,0,0,2,0,...,0,0,1,0,0,0,0,0,0,433
5,0,0,4,12,12,51,0,0,0,0,...,0,0,0,0,0,0,0,0,0,78
6,0,0,0,0,0,0,271,34,0,0,...,0,0,0,0,0,0,0,0,0,336
7,2,0,0,0,0,0,195,58,0,0,...,0,0,0,0,0,0,0,0,0,416
8,6,0,0,0,3,0,0,0,100,53,...,0,0,3,2,0,0,0,0,0,465


In [205]:
predictions = model.predict(X_test)
predictions2=pd.DataFrame(predictions)
predictions3=np.argmax(predictions,axis=1)
X_test2=pd.DataFrame(X_test)
X_test2['prediction']=predictions3
X_test2['final_subcat']=y_test
X_test2.index=x_test.index
X_test2['article']=x_test['article']

In [206]:
tt6.columns

Index([     'article',              0,              1,              2,
                    3,              4,              5,              6,
                    7,              8,
       ...
                   96,             97,             98,             99,
             'subcat',       'cheese',  'canned_fish',        'chips',
       'final_subcat',    'make_code'],
      dtype='object', length=107)

In [207]:
subcat_index_mapping=tt6[['make_code','subcat']].drop_duplicates()

In [208]:
subcat_index_mapping=subcat_index_mapping.reset_index()
subcat_index_mapping=subcat_index_mapping.drop(['index'],axis=1)

In [209]:
subcat_index_mapping[subcat_index_mapping['make_code']==1]

,make_code,subcat
28,1,BABY - BASICS


In [210]:
X_test2['correct']=np.where(X_test2['final_subcat']==X_test2['prediction'],1,0)
np.mean(X_test2['correct'])

0.658918516944651

In [211]:
X_test2=X_test2.reset_index()
X_test2=X_test2.drop(['index'],axis=1)

In [212]:
X_test2.index

RangeIndex(start=0, stop=94130, step=1)

In [213]:
X_test2=pd.merge(X_test2,subcat_index_mapping,left_on='final_subcat',right_on='make_code',how='inner')

In [214]:
X_test2.columns

Index([             0,              1,              2,              3,
                    4,              5,              6,              7,
                    8,              9,
       ...
                   96,             97,             98,             99,
         'prediction', 'final_subcat',      'article',      'correct',
          'make_code',       'subcat'],
      dtype='object', length=106)

In [215]:
X_test2=X_test2.rename(columns={"subcat":"final_subcat_name"})

In [216]:
X_test2=pd.merge(X_test2,subcat_index_mapping,left_on='prediction',right_on='make_code',how='inner')
X_test2=X_test2.rename(columns={"subcat":"predicted_subcat_name"})

In [217]:
xxxx=np.argmax(model.predict((w2v_model['lamb'].reshape(-1,100)+w2v_model['heart'].reshape(-1,100))/2),axis=1)
subcat_index_mapping[subcat_index_mapping['make_code']==xxxx[0]]

,make_code,subcat
0,110,unknown


In [218]:
X_test2[(X_test2['correct']==0) & (X_test2['final_subcat_name'].str.contains("CHEESE")) & (X_test2['predicted_subcat_name'].str.contains("CHEESE"))][['article','final_subcat_name','predicted_subcat_name']]

,article,final_subcat_name,predicted_subcat_name
83,boosey oma fav spiced gouda,CHEESE PRE-PACKED ENTERTAINING,DAIRY - CHEESE
84,kapiti gouda garlic chive,CHEESE PRE-PACKED ENTERTAINING,DAIRY - CHEESE
85,chrystal trio cheese selction,CHEESE PRE-PACKED ENTERTAINING,DAIRY - CHEESE
86,south cape cheese gouda block,CHEESE PRE-PACKED ENTERTAINING,DAIRY - CHEESE
87,biodynamic organic cheddar,CHEESE PRE-PACKED ENTERTAINING,DAIRY - CHEESE
88,warrnambool cheddar matured classic,CHEESE PRE-PACKED ENTERTAINING,DAIRY - CHEESE
89,south cape cheese sundried tomato,CHEESE PRE-PACKED ENTERTAINING,DAIRY - CHEESE
90,brancourt cheese colby,CHEESE PRE-PACKED ENTERTAINING,DAIRY - CHEESE
91,puhoi valley creamy blue,CHEESE PRE-PACKED ENTERTAINING,DAIRY - CHEESE
92,castello cheese blue slouse,CHEESE PRE-PACKED ENTERTAINING,DAIRY - CHEESE


In [219]:
y=pd.crosstab(X_test2['final_subcat_name'],X_test2['predicted_subcat_name'])

In [220]:
y.head()

predicted_subcat_name,ASIAN FOODS,BABY - BASICS,BABY - CARE,BABY - NAPPIES,BABY FOOD,BABY FORMULA,BEEF,BEEF CASE READY,BISCUITS - CRISPBREAD & CRACKER,BISCUITS - PLAIN & FANCY,...,SOFT DRINKS - WATER,SOUPS,SPREADS - HONEY,SPREADS - JAM,SPREADS - OTHER,SPREADS - PEANUT BUTTER,SUGAR,SUSHI,TEA,unknown
final_subcat_name,,,,,,,,,,,,,,,,,,,,,
ASIAN FOODS,172,0,0,0,4,0,0,0,1,4,...,2,2,0,3,0,0,3,8,5,488
BABY - BASICS,0,210,17,10,15,2,0,0,0,0,...,2,0,0,0,0,0,0,0,0,806
BABY - CARE,0,8,106,16,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,134
BABY - NAPPIES,0,3,2,227,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,48
BABY FOOD,1,3,3,1,432,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,171


In [221]:
y.to_csv("//QATLPCFS001/Users/akishore/Desktop/DL/voc/prediction_pivot14.csv")

In [222]:
comment="didnt have enough feta cheese spread lights"
bigram[preprocess(comment).split()]

C:\Program Files\Anaconda3\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['didnt', 'enough', 'fetum', 'cheese', 'spread', 'light']

In [223]:
j=3
x=bigram[(preprocess(comments.loc[j][0])).split()]
print(x)

['fetum', 'cheese']


C:\Program Files\Anaconda3\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [224]:
my_index=np.argmax(model.predict(w2v_model['john_west'].reshape(-1,100)))

In [225]:
subcat_index_mapping[subcat_index_mapping['make_code']==my_index]

,make_code,subcat
4,14,CANNED FISH


In [226]:
comments.head()

,Comment
0,mainland cheddar available
1,cholesterol reducing cheese slice
2,didnt enough kraft cheese spread light
3,fetum cheese
4,toffutti cream cheese deleted


In [227]:
# pred=[]
# pred_score=[]
# for j in range(comments.shape[0]):
#     x=bigram[(preprocess(comments.loc[j][0])).split()]
#     y=[0,0]
#     k=0
#     m=0
#     z2=0
#     if(j%100==0):
#         print(j)
#         print(len(pred))
#     if(len(x)>0):
        
#         for i in range(len(x)):
#             try:
#                 z2=z2+w2v_model[x[i]].reshape(-1,100)
#                 #y=y+model2.predict_proba(model[x[i]].reshape(-1,100))
#                 m=m+1
#             except KeyError:
#                 k=k+1
    
#         if(m>0):
#             z=model.predict(z2/m)
#         else:
#             z=[0]
#         #print(comments.loc[j][0])
#     else:
#         z=[0]
#     try:
        

#         pred.append(subcat_index_mapping[subcat_index_mapping['make_code']==np.argmax(z,axis=1)[0]]['subcat'].item())
#         #print(subcat_index_mapping[subcat_index_mapping['make_code']==np.argmax(z,axis=1)[0]]['subcat'].item())
#         pred_score.append(z[0][np.argmax(z,axis=1)][0])
       
#     except:
#         pred.append("unknown2")
#         pred_score.append(0)


C:\Program Files\Anaconda3\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


0
0
100
100
200
200
300
300
400
400
500
500
600
600
700
700
800
800
900
900
1000
1000
1100
1100
1200
1200
1300
1300
1400
1400
1500
1500
1600
1600
1700
1700
1800
1800
1900
1900
2000
2000
2100
2100
2200
2200
2300
2300
2400
2400
2500
2500
2600
2600
2700
2700
2800
2800
2900
2900
3000
3000
3100
3100
3200
3200
3300
3300
3400
3400
3500
3500
3600
3600
3700
3700
3800
3800
3900
3900
4000
4000
4100
4100
4200
4200
4300
4300
4400
4400
4500
4500
4600
4600
4700
4700
4800
4800
4900
4900
5000
5000
5100
5100
5200
5200
5300
5300
5400
5400
5500
5500
5600
5600
5700
5700
5800
5800
5900
5900
6000
6000
6100
6100
6200
6200
6300
6300
6400
6400
6500
6500
6600
6600
6700
6700
6800
6800
6900
6900
7000
7000
7100
7100
7200
7200
7300
7300
7400
7400
7500
7500
7600
7600
7700
7700
7800
7800
7900
7900
8000
8000
8100
8100
8200
8200
8300
8300
8400
8400
8500
8500
8600
8600
8700
8700
8800
8800
8900
8900
9000
9000
9100
9100
9200
9200
9300
9300
9400
9400
9500
9500
9600
9600
9700
9700
9800
9800
9900
9900
10000
10000
10100
10100


In [977]:
# len(pred)

69045

In [978]:
# pred_score[0:5]

[0.473746, 0.29134691, 0.48425904, 0.40222928, 0.34226823]

In [979]:
# pred[0:5]

['DAIRY - CHEESE',
 'DAIRY - CHEESE',
 'DAIRY - BUTTER & MARGARINE',
 'CHEESE PRE-PACKED COOKING',
 'DAIRY - CHEESE']

In [228]:
comments=pd.read_table("//QATLPCFS001/Users/akishore/Desktop/DL/voc/comments.csv",encoding='latin1')
# comments['predicted_subcat']=pred
# comments['predicted_subcat_score']=pred_score


In [229]:
z=model.predict(w2v_model['milk'].reshape(-1,100))
subcat_index_mapping[subcat_index_mapping['make_code']==np.argmax(z,axis=1)[0]]['subcat'].item()

'DAIRY - MILK'

In [230]:
comments.head()

,Comment
0,Mainland cheddar not available
1,no cholesterol reducing cheese slices
2,didnt have enough Kraft cheese spread light
3,No Feta cheese.
4,Toffutti cream cheese has been deleted


In [231]:
zz="Fruits and vegetables I was looking for were not there..possibly run out"
xx=bigram[preprocess(zz).split()]

C:\Program Files\Anaconda3\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [232]:
# pred_subcat2=[]
# for j in range(comments.shape[0]):
#     xx=bigram[(preprocess(comments.loc[j][0])).split()]
#     y=("fruit" in xx) + ("veg" in xx) + ("vegetable" in xx) 
#     z=("banana" in xx)
#     bbq=("bbq" in xx) + ("chicken" in xx)
#     meat = ("meat" in xx)
#     if(j%100==0):
#         print(j)
#     if (y>0):
#         pred_subcat2.append('CANNED VEGETABLES')
#     elif (z==1):    
#         pred_subcat2.append('PRODUCE')
#     elif(bbq==2):
#         pred_subcat2.append('SERVICED MEAT')
#     elif (meat==1):
#         pred_subcat2.append('SERVICED MEAT')
#     else:
#         pred_subcat2.append(comments.loc[j]['predicted_subcat'])

In [233]:
# pred_subcat2[280]

In [986]:
# comments=comments.drop(['predicted_subcat'],axis=1)
# comments['predicted_subcat']=pred_subcat2

In [234]:
#comments.to_csv("//QATLPCFS001/Users/akishore/Desktop/DL/voc/comments_predicted_subcats22.csv",index=False)

In [235]:
from nltk import tokenize

In [236]:
comments=pd.read_table("//QATLPCFS001/Users/akishore/Desktop/DL/voc/comments.csv",encoding='latin1')

In [237]:
final_pred=[]
final_pred_score=[]
for j in range(comments.shape[0]):
    pred=[]
    pred_score=[]
    #j=269
    xx=re.split(r'[.,;!\n]', comments.loc[j][0])
    
    for k in range(len(xx)):
        x=bigram[(preprocess(xx[k])).split()]
        #print(x)
        y=[0,0]
        k=0
        m=0
        z2=0
        yy=("fruit" in x) + ("veg" in x) + ("vegetable" in x) 
        z=("banana" in x)
        bbq=("bbq" in x) + ("chicken" in x)
        meat = ("meat" in x)
        organic = ("organic" in x)
        chicken=("chicken" in x)
        deli = ("deli" in x)
        rock = ("rock" in x)
        seafood = ("seafood" in x)
        
        if (yy>0):
            pred.append('PRODUCE')
        elif (z==1):    
            pred.append('PRODUCE')
        elif(bbq==2):
            pred.append('SERVICED MEAT')
        elif (meat==1):
            pred.append('SERVICED MEAT')
        elif(organic==1):
            pred.append('ORGANICS')
        elif(chicken==1):
            pred.append('FRESH POULTRY')
        elif((deli==1) and (rock==0)):
            pred.append('DELI')
        elif(seafood==1):
            pred.append('SEAFOOD')
        else:
            #pred_subcat2.append(comments.loc[j]['predicted_subcat'])
            if(j%100==0):
                print(j)
                #print(len(pred))
            if(len(x)>0):

                for i in range(len(x)):
                    try:
                        z2=z2+w2v_model[x[i]].reshape(-1,100)
                        #y=y+model2.predict_proba(model[x[i]].reshape(-1,100))
                        m=m+1
                        #print(m)
                    except KeyError:
                        k=k+1

                if(m>0):
                    z=model.predict(z2/m)
                else:
                    z=[0]
                #print(comments.loc[j][0])
            else:
                z=[0]
        #print(z)
        try:


            pred.append(subcat_index_mapping[subcat_index_mapping['make_code']==np.argmax(z,axis=1)[0]]['subcat'].item())
            #print(subcat_index_mapping[subcat_index_mapping['make_code']==np.argmax(z,axis=1)[0]]['subcat'].item())
            #print(subcat_index_mapping[subcat_index_mapping['make_code']==np.argmax(z,axis=1)[0]]['subcat'].item())
            if(subcat_index_mapping[subcat_index_mapping['make_code']==np.argmax(z,axis=1)[0]]['subcat'].item()=="unknown"):
                pred_score.append(0.01)
            else:

                pred_score.append(z[0][np.argmax(z,axis=1)][0])

        except:
            pred.append("unknown2")
            pred_score.append(0)
    #print(pred)
    xxx=', '.join(set([word for word in pred]))
    yyy=', '.join(set([str(word) for word in pred_score if word>0.0]))
    final_pred.append(xxx)
    final_pred_score.append(yyy)


C:\Program Files\Anaconda3\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


0
100
100
200
200
200
200
300
300
300
300
300
400
400
400
400
400
400
400
400
400
500
500
600
600
600
600
700
700
700
700
700
700
700
700
700
700
700
700
800
800
800
900
900
900
1000
1200
1300
1300
1300
1400
1400
1400
1400
1500
1500
1500
1600
1700
1700
1700
1700
1800
1800
1800
1800
1800
1900
2000
2000
2100
2100
2300
2300
2300
2300
2400
2400
2500
2500
2500
2500
2500
2600
2600
2600
2600
2700
2800
2800
2800
2900
2900
3000
3100
3200
3200
3300
3300
3400
3400
3400
3500
3600
3700
3800
3900
4000
4100
4100
4200
4200
4300
4300
4300
4400
4400
4400
4400
4400
4500
4500
4500
4600
4700
4700
4700
4800
4800
4900
5000
5100
5100
5200
5300
5400
5400
5400
5500
5500
5600
5600
5700
5700
5800
5800
5800
5800
5900
6000
6000
6000
6000
6000
6000
6100
6100
6100
6200
6200
6300
6400
6400
6400
6500
6500
6500
6600
6600
6700
6800
6800
6800
6800
6900
6900
7000
7000
7000
7100
7200
7300
7400
7400
7400
7400
7400
7400
7400
7500
7600
7800
7800
7900
8000
8100
8100
8200
8300
8300
8500
8500
8500
8600
8600
8700
8800
8900
8900
89

In [238]:
comments=pd.read_table("//QATLPCFS001/Users/akishore/Desktop/DL/voc/comments.csv",encoding='latin1')
comments['predicted_subcat']=final_pred
comments['predicted_subcat_score']=final_pred_score
comments.to_csv("//QATLPCFS001/Users/akishore/Desktop/DL/voc/comments_predicted_subcats36.csv",index=False)

In [239]:
comments.head()

,Comment,predicted_subcat,predicted_subcat_score
0,Mainland cheddar not available,DAIRY - CHEESE,0.525254
1,no cholesterol reducing cheese slices,DAIRY - CHEESE,0.382147
2,didnt have enough Kraft cheese spread light,DAIRY - BUTTER & MARGARINE,0.584017
3,No Feta cheese.,"unknown2, CHEESE PRE-PACKED COOKING",0.367359
4,Toffutti cream cheese has been deleted,DAIRY - CHEESE,0.409461


In [991]:
w2v_model.save("word2vec_final_voc")

2017-11-08 10:24:14,888 : INFO : saving Word2Vec object under word2vec_final_voc, separately None
2017-11-08 10:24:14,898 : INFO : not storing attribute syn0norm
2017-11-08 10:24:14,899 : INFO : not storing attribute cum_table
2017-11-08 10:24:14,989 : INFO : saved word2vec_final_voc


In [992]:
model.save('my_model_voc.h5')

In [1039]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(min_df=.0025, max_df=.1, stop_words='english')
tvec_weights = tvec.fit_transform(data['ARTICLE_NAME'])
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
64,dvd,0.024516
34,card,0.015206
39,chicken,0.014172
17,black,0.013914
38,cheese,0.012828
12,beef,0.011941
20,blue,0.010523
16,bk,0.010258
156,red,0.010113
201,white,0.010015


In [1040]:
weights_df=weights_df.reset_index()
weights_df=weights_df.drop(['index'],axis=1)

In [1135]:
x="organic"
data[data['ARTICLE_NAME'].str.contains('fruit') & data['ARTICLE_NAME'].str.contains('veg')]

,ARTICLE_NAME,SUBCAT_NAME,text
7130,heinz lk shredz fruit veg,BABY FOOD,gc baby apple custad age golden circle baby fd...
8174,vegetali fruit caramelsed pear,DAIRY - SNACKS KIDS,weight watcher lemon deliciou yoplait blueberr...
8522,nippy fruit nippy frveg,DAIRY - CHILLED JUICES & DRINKS,lt apple juice supreme orange nectar juice chi...
13900,juice fruit veg citru splash,FRUIT JUICE - LONG LIFE,dole juice mango mambo juce strawberry kiwifru...
16120,vegetali fruit rhubarb blubery,DAIRY - SNACKS KIDS,weight watcher lemon deliciou yoplait blueberr...
17056,gc fruit veg jc pinepunch mnth,BABY FOOD,gc baby apple custad age golden circle baby fd...
21426,fruit veg juice apple plum fusn,FRUIT JUICE - LONG LIFE,dole juice mango mambo juce strawberry kiwifru...
21680,fruit veg breakfast,FRUIT JUICE - LONG LIFE,dole juice mango mambo juce strawberry kiwifru...
29709,artisse fruit veg bar app rspbry,HEALTH FOODS,sunrice brown rice chip parm tom macro apr cur...
34540,artisse fruit veg bar app raisin,HEALTH FOODS,sunrice brown rice chip parm tom macro apr cur...


In [1058]:
article_count=[]
article_subcat_name=[]
subcat_count=[]
subcat_name=[]
for i in range(weights_df.shape[0]):
    print(i)
    x=weights_df.loc[i]['term']
    #print(x)
    art_len=len(data[data['ARTICLE_NAME'].str.contains(x.lower())]['SUBCAT_NAME'].value_counts())
    subcat_len=len(data[data['SUBCAT_NAME'].str.contains(x.upper())]['SUBCAT_NAME'].value_counts())
    if(art_len>0):
        
        article_count.append(data[data['ARTICLE_NAME'].str.contains(x.lower())]['SUBCAT_NAME'].value_counts()[0])
        article_subcat_name.append(data[data['ARTICLE_NAME'].str.contains(x.lower())]['SUBCAT_NAME'].value_counts().index[0])
        
    else:
        article_count.append(0)
        article_subcat_name.append('unknown')
    
    if(subcat_len>0):
        
        subcat_count.append(data[data['SUBCAT_NAME'].str.contains(x.upper())]['SUBCAT_NAME'].value_counts()[0])
        subcat_name.append(data[data['SUBCAT_NAME'].str.contains(x.upper())]['SUBCAT_NAME'].value_counts().index[0])
        
    else:
        subcat_count.append(0)
        subcat_name.append('unknown')
    #subcat_count.append(data[data['SUBCAT_NAME'].str.contains(x.upper())]['SUBCAT_NAME'].value_counts()[0])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209


In [1054]:
len(article_count)

210

In [1057]:
data[data['ARTICLE_NAME'].str.contains(x.lower())]['SUBCAT_NAME'].value_counts().index[0]

Index(['DAIRY - YOGHURT', 'unknown', 'FREEZER - ICE CREAM TAKE HOME',
       'DAIRY - SNACKS KIDS', 'MUESLI BARS', 'HEALTH FOODS', 'DAIRY - DIPS',
       'BISCUITS - PLAIN & FANCY', 'DELI - CHILLED', 'SNACK - NUTS',
       'DAIRY - QUICK & EASY MEALS', 'DAIRY - CHILLED JUICES & DRINKS',
       'FREEZER - ICE CREAM SINGLES', 'BABY FOOD',
       'FREEZER - DESSERTS & PASTRY', 'BREAKFAST - MUESLI & OATS',
       'FREEZER - ICE CREAM MULTIPACKS', 'CONFECTIONERY - SUGAR',
       'CONFECTIONERY - NOVELTY', 'DAIRY - SNACKS ADULT', 'DAIRY - CHEESE',
       'CHEESE PRE-PACKED ENTERTAINING', 'PETFOOD', 'SALADS PREPACKED',
       'RESALE CAKE'],
      dtype='object')

In [1125]:
data[data['ARTICLE_NAME'].str.contains("chicken")]['SUBCAT_NAME'].value_counts()

unknown                            817
FRESH POULTRY                      808
HOT FOOD                           332
FREEZER - POULTRY                  274
FREEZER - MEALS                    186
PET NEEDS - CAT FOOD WET           146
DAIRY - QUICK & EASY MEALS         144
DAIRY - SMALLGOODS                 118
SUSHI                              108
NOODLES                             99
PREPARED MEALS                      67
CHIPS - SHARING                     67
PET NEEDS - DOG FOOD WET            58
PET NEEDS - DOG FOOD DRY            56
PET NEEDS - DOG TREATS              50
SOUPS                               48
PETFOOD                             45
ASIAN FOODS                         35
BISCUITS - SNACKING                 33
BABY FOOD                           33
PET NEEDS - CAT FOOD DRY            32
SALADS PREPACKED                    27
CANNED FISH                         20
CHEESE PRE-PACKED ENTERTAINING      16
CHIPS - MULTIPACKS                  15
FREEZER - PIZZA          

In [1129]:
np.argmax(model.predict(w2v_model['chicken'].reshape(-1,100)))

110

In [1022]:
data[data['SUBCAT_NAME'].str.contains("ORGANIC")]

,ARTICLE_NAME,SUBCAT_NAME,text
43,macro organic beef blade steak,ORGANIC MEAT,macro organic beef blade steak belmore organic...
436,org nectarine white,ORGANICS,org nectarine white org garlic org pumpkin jar...
488,org garlic,ORGANICS,org nectarine white org garlic org pumpkin jar...
736,org pumpkin jarrahdal,ORGANICS,org nectarine white org garlic org pumpkin jar...
751,belmore organic rib eye steak rw,ORGANIC MEAT,macro organic beef blade steak belmore organic...
805,macro organic lamb forequarterchop,ORGANIC MEAT,macro organic beef blade steak belmore organic...
883,org carrot petite,ORGANICS,org nectarine white org garlic org pumpkin jar...
1035,org leek,ORGANICS,org nectarine white org garlic org pumpkin jar...
1231,org lebanese cucumber,ORGANICS,org nectarine white org garlic org pumpkin jar...
1234,organic beef sandwchsteak,ORGANIC MEAT,macro organic beef blade steak belmore organic...


In [1021]:
weights_df['term']

0          almond
1           apple
2         apricot
3        assorted
4            baby
5           bacon
6             bag
7            ball
8          banana
9             bar
10            bbq
11           bean
12           beef
13           beer
14          berry
15           bird
16             bk
17          black
18            blk
19          block
20           blue
21           bond
22           book
23         bottle
24           bowl
25            boy
26            bra
27          bread
28          brown
29         butter
          ...    
180           soy
181         spray
182          star
183         steak
184    strawberry
185         style
186         sugar
187         super
188         sweet
189            sz
190           tea
191           tee
192           tom
193        tomato
194          tuna
195        turkey
196        valley
197       vanilla
198           veg
199         water
200         watty
201         white
202          wing
203           wow
204       

In [1007]:
data['text'] = data[['ARTICLE_NAME','SUBCAT_NAME']].groupby(['SUBCAT_NAME'])['ARTICLE_NAME'].transform(lambda x: ' '.join(x))

In [1008]:
data.head()

,ARTICLE_NAME,SUBCAT_NAME,text
0,yoplait berry delight,DAIRY - YOGHURT,yoplait berry delight bulla greek style honey ...
1,destiny playstatpre order,PET NEEDS - DOG TREATS,destiny playstatpre order totalcare dog festiv...
2,disney plush,BABY - BASICS,disney plush bodysuit qa pilcher sz sleepsuit ...
3,gold alula newborn,BABY FORMULA,gold alula newborn fresco goat toddler formula...
4,rowy caramel fudge,CHEESE PRE-PACKED ENTERTAINING,rowy caramel fudge puhoi cheese brie karikaa c...


In [1009]:
data2=data[['SUBCAT_NAME','text']].drop_duplicates()

In [1014]:
data2.shape

(111, 2)

In [1059]:
tvec

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.1, max_features=None, min_df=0.0025,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [1100]:
data2.shape

(111, 2)

In [1108]:
tvec_weights.shape

(111, 58572)

In [1111]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(min_df=.001, max_df=0.9,max_features=100000)
tvec_weights = tvec.fit_transform(data2['text'])
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

KeyError: 'weight'

In [1105]:
weights_df[weights_df['term']=='original']

,term,weight
36580,original,0.007841


In [1112]:
len(data2[data2['text'].str.contains("organic")]['SUBCAT_NAME'].value_counts())

74

In [1117]:
data.loc[data[data['ARTICLE_NAME'].str.contains("organic")].index]['SUBCAT_NAME'].value_counts()

HEALTH FOODS                       370
unknown                            332
BABY FOOD                          261
ORGANIC MEAT                       160
TEA                                 96
DAIRY - YOGHURT                     79
DAIRY - QUICK & EASY MEALS          61
COFFEE                              55
RESALE BREAD                        54
HAIR CARE                           52
ORGANICS                            47
PASTA                               44
FRUIT JUICE - LONG LIFE             41
ASIAN FOODS                         35
DAIRY - MILK                        35
DAIRY - CHEESE                      32
FREEZER - VEGETABLES                31
SPREADS - HONEY                     30
LONGLIFE MILK & SOY DRINKS          29
BREAKFAST - MUESLI & OATS           28
CANNED VEGETABLES                   25
DAIRY - CHILLED JUICES & DRINKS     25
FRESH POULTRY                       24
RICE                                24
DAIRY - DIPS                        23
PASTA SAUCE & CHEESE     

In [1091]:
data2[data2['text'].str.contains("original")]

,SUBCAT_NAME,text
0,DAIRY - YOGHURT,yoplait berry delight bulla greek style honey ...
1,PET NEEDS - DOG TREATS,destiny playstatpre order totalcare dog festiv...
3,BABY FORMULA,gold alula newborn fresco goat toddler formula...
4,CHEESE PRE-PACKED ENTERTAINING,rowy caramel fudge puhoi cheese brie karikaa c...
5,RESALE CAKE,cookie couture iced gingerbread tea bulk mini ...
6,HAIR CARE,pear conditioner clarify joico moisture recovr...
9,FREEZER - ICE CREAM MULTIPACKS,lr multipack pop refresher peter choc wedge sp...
11,ASIAN FOODS,marion korn chili ssm str fry suc bond frnt tr...
13,DAIRY - QUICK & EASY MEALS,raguletto spin ric agn bazaar garlic bread che...
14,PET NEEDS - CAT FOOD WET,fancy feast tndrturkey rice tscny wow cat chk ...
